<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction-to-Cross-Validation---Lab" data-toc-modified-id="Introduction-to-Cross-Validation---Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction to Cross-Validation - Lab</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Let's-get-started" data-toc-modified-id="Let's-get-started-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Let's get started</a></span><ul class="toc-item"><li><span><a href="#Train-test-split" data-toc-modified-id="Train-test-split-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Train-test split</a></span></li><li><span><a href="#Fit-the-model" data-toc-modified-id="Fit-the-model-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Fit the model</a></span></li><li><span><a href="#Residuals-and-MSE" data-toc-modified-id="Residuals-and-MSE-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Residuals and MSE</a></span></li></ul></li><li><span><a href="#Cross-Validation:-let's-build-it-from-scratch!" data-toc-modified-id="Cross-Validation:-let's-build-it-from-scratch!-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Cross-Validation: let's build it from scratch!</a></span><ul class="toc-item"><li><span><a href="#Create-a-cross-validation-function" data-toc-modified-id="Create-a-cross-validation-function-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Create a cross-validation function</a></span></li><li><span><a href="#Apply-it-to-the-Boston-Housing-data" data-toc-modified-id="Apply-it-to-the-Boston-Housing-data-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Apply it to the Boston Housing data</a></span></li><li><span><a href="#Perform-a-linear-regression-for-each-fold-and-calculate-the-training-and-test-error" data-toc-modified-id="Perform-a-linear-regression-for-each-fold-and-calculate-the-training-and-test-error-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Perform a linear regression for each fold and calculate the training and test error</a></span></li></ul></li><li><span><a href="#Cross-Validation-using-Scikit-Learn" data-toc-modified-id="Cross-Validation-using-Scikit-Learn-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Cross-Validation using Scikit-Learn</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Summary</a></span></li></ul></li></ul></div>

# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to pre-process below.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features['B']
logdis = np.log(boston_features['DIS'])
loglstat = np.log(boston_features['LSTAT'])

# Min-Max scaling
boston_features['B'] = (b-min(b))/(max(b)-min(b))
boston_features['DIS'] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# Standardization
boston_features['LSTAT'] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [12]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns = ['target'])
type(X)

pandas.core.frame.DataFrame

### Train-test split

Perform a train-test split with a test set of 20%.

In [13]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [14]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [15]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [16]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

linreg.fit(X_train, y_train)
y_hat_test = linreg.predict(X_test)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [17]:
from sklearn.metrics import mean_squared_error
test_residuals = y_hat_test - y_test

test_mse = mean_squared_error(y_test, y_hat_test)
test_mse

23.7941900777502

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [18]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    data = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations//k
    leftovers = num_observations%k
    folds = []
    start_obs = 0
    for fold_n in range(1,k+1):
        if fold_n <= leftovers:
            #Fold Size will be 1 larger to account for leftovers
            fold = data.iloc[start_obs : start_obs+fold_size+1]
            folds.append(fold)
            start_obs += fold_size + 1
        else:
            fold = data.iloc[start_obs : start_obs+fold_size]
            folds.append(fold)
            start_obs += fold_size
    # add 1 to fold size to account for leftovers           
    return folds

### Apply it to the Boston Housing data

In [19]:
# Make sure to concatenate the data again
bos_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [20]:
# Apply kfolds() to bos_data with 5 folds
bos_folds = kfolds(bos_data, 5)

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [21]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(bos_folds) if i!=n])
    test = bos_folds[n]
    # Fit a linear regression model
    linreg.fit(train[X.columns], train[y.columns])
    # Evaluate Train and Test errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    train_residuals = y_hat_train - train[y.columns]
    test_residuals = y_hat_test - test[y.columns]
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))
print(train_errs)
print(test_errs)

[target    24.195577
dtype: float64, target    23.032087
dtype: float64, target    19.745073
dtype: float64, target    15.317101
dtype: float64, target    22.329973
dtype: float64]
[target    13.405145
dtype: float64, target    17.444017
dtype: float64, target    37.032711
dtype: float64, target    58.279544
dtype: float64, target    26.097989
dtype: float64]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [23]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring='neg_mean_squared_error')

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [26]:
np.mean(cv_5_results)

-30.451881143540326

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 